In [1]:
import numpy as np
from math import round

In [2]:
input_params = {
"k": 115740740740,
"lambda": 1490000000000000000,
"delta": 7425000000000000,
"capPayoff": 10000000000000000000,
"capNotional": 60000000000000000000000,
"capLeverage": 5000000000000000000,
"circuitBreakerWindow": 2592000,
"circuitBreakerMintTarget": 5000000000000000000000,
"maintenanceMarginFraction": 40000000000000000,
"maintenanceMarginBurnRate": 50000000000000000,
"liquidationFeeRate": 50000000000000000,
"tradingFeeRate": 750000000000000,
"minCollateral": 100000000000000,
"priceDriftUpperLimit": 87000000000000,
"averageBlockTime": 0
}

## Explanations for formula based features

In [10]:
print(f'''
Funding rate per day: {round(input_params['k'] * 2 * 86400 * 100/1e18, 2)}%
Static bid-ask spread: {round((np.exp(input_params['delta']/1e18) - np.exp(-input_params['delta']/1e18)) * 100, 2)}%
Market Impact (for trade with size 1% of OI cap): {round((np.exp(input_params['lambda'] * 0.01/1e18)-1) * 100, 2)}%
OI cap as percentage of 8mn OVL: {round(input_params['capNotional'] * 100/(1e18 * 8_000_000), 2)}%
Max allowed price change of longer TWAP (3600s): {round((np.exp(input_params['priceDriftUpperLimit']*3600/1e18) - 1) * 100, 2)}%
'''
)


Funding rate per day: 2.0%
Static bid-ask spread: 1.49%
Market Impact (for trade with size 1% of OI cap): 1.5%
OI cap as percentage of 8mn OVL: 0.75%
Max allowed price change of longer TWAP (3600s): 36.78%

